# Uploadinf data in a GS using python
## Set up
1. Loading packages
2. Loading credentials
3. Open the GS

## Uploading data from a CSV file without modifications in a Google Sheet worksheet
4. Reading file
5. uploading data in a GS

## Upldoading data into specific GS-cells and preprocess data
### Preprocessing data
6. Reading file
### Updating data

### Set up

In [175]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import pandas as pd
import numpy as np
import ast

In [1]:
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive",
]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "client_secret_cred.json", scope
)

In [235]:
client = gspread.authorize(credentials)
# open the GS
spreadsheet = client.open("data_nd_business")
worksheet = spreadsheet.worksheet("sheet1")

### Uploading data from a CSV file without modifications in a Google Sheet worksheet

In [ ]:
# with open("original_data_1.csv", "r") as f:
#     content = f.read()
# %%capture
# client.import_csv(spreadsheet.id, data=content)

In [269]:
df = pd.read_csv("original_data_1.csv", sep=',')
df = df.replace(np.nan, None)
content=[list(df.columns)]
content.extend(df.values.tolist())

In [270]:
worksheet.update(content)

{'spreadsheetId': '1j-gYHSeBScFURw8oo_79CFOLqDsKMoMBE6rOB5F_tSk',
 'updatedRange': 'sheet1!A1:G148',
 'updatedRows': 148,
 'updatedColumns': 7,
 'updatedCells': 739}

### Upldoading data into specific GS-cells and preprocess data 

__Reading CSV file__ with data to fill out the template

In [311]:
df_to_fill_template = pd.read_csv("original_data_1.csv")
df_to_fill_template.head()

,id,id_business,commercial_registered_agent,registered_agent,owners,business_name,status
0,322123,2,NaN,NaN,"Elite Golf, LLC","['X Golf Minot', 'Trade Name']",Active
1,336584,3,NaN,NaN,"Elite Golf 2, LLC","['X Golf Bismarck', 'Trade Name']",Active
2,144321,4,CORPORATION SERVICE COMPANY\n418 N 2ND ST\nBIS...,NaN,NaN,"['ATOS HEALTHCARE SERVICES, LLC', 'Limited Lia...",Active
3,349089,1,NaN,"ZANE SCHOBINGER\n800 11TH AVE W\nDICKINSON, N...",NaN,"['XYZ Services LLC', 'Limited Liability Compan...",Active
4,111130,3,C T CORPORATION SYSTEM\n120 W SWEET AVE\nBISMA...,NaN,NaN,"['XYLEM WATER SOLUTIONS U.S.A., INC.', 'Corpor...",Active


In [312]:
print("################# unique", df_to_fill_template.id.value_counts)
print("################# INFO")
print(df_to_fill_template.info())

################# unique <bound method IndexOpsMixin.value_counts of 0      322123
1      336584
2      144321
3      349089
4      111130
        ...  
142    103679
143    352364
144    144320
145    319429
146    162412
Name: id, Length: 147, dtype: int64>
################# INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           147 non-null    int64 
 1   id_business                  147 non-null    int64 
 2   commercial_registered_agent  65 non-null     object
 3   registered_agent             49 non-null     object
 4   owners                       30 non-null     object
 5   business_name                147 non-null    object
 6   status                       147 non-null    object
dtypes: int64(2), object(5)
memory usage: 8.2+ KB
None


In [313]:
df_to_fill_template = df_to_fill_template.replace(np.nan, None)

__Reading data from a template__ GS worksheet

In [314]:
worksheet_template = spreadsheet.worksheet("data_nd_business")

In [315]:
df_template = pd.DataFrame(worksheet_template.get_all_records())
df_template.head()

,id,id_business,commercial_registered_agent,registered_agent,owners,business_name,status
0,105993,1,,,,,
1,289003,2,,,,,
2,329549,3,,,,,
3,357555,4,,,,,
4,103327,1,,,,,


__Creating a DataFrame by combining the template and the data from the CSV file__ df_to_fill_template and df_template

In [316]:
data_df_to_fill_template = df_to_fill_template.set_index(["id", "id_business"])
data_df_template = df_template.set_index(["id", "id_business"])


In [318]:
data = data_df_to_fill_template.loc[data_df_template.index,data_df_template.columns]
data = data.reset_index()

In [321]:
# creating a new worksheet
date = datetime.today().strftime("%Y-%m-%d")
a,b = data.shape
worksheet = spreadsheet.add_worksheet(title=date,rows=a, cols=b)

In [322]:
data = data.replace(np.nan, None)
content=[list(data.columns)]
content.extend(data.values.tolist())

worksheet.update(content)

{'spreadsheetId': '1j-gYHSeBScFURw8oo_79CFOLqDsKMoMBE6rOB5F_tSk',
 'updatedRange': "'2024-10-09'!A1:G148",
 'updatedRows': 148,
 'updatedColumns': 7,
 'updatedCells': 739}

In [151]:
%%capture
client.import_csv(spreadsheet.id, data=com_reg_agent_string)

In [180]:
col_filter= ["commercial_registered_agent", "registered_agent", "owners"]
worksheet= spreadsheet.worksheet("test_1")
for col in col_filter:
    temp_df = df.loc[df[col].notnull()]
    temp_data = temp_df.values.tolist()
    
    # worksheet.append_rows(temp_data)
temp_df.head(5)    

,commercial_registered_agent,registered_agent,owners,id,business_name,status
6,None,None,Oech Foods LLC,352364,"['X FOODS', 'Trade Name']",Active
10,None,None,"Elite Golf, LLC",322123,"['X Golf Minot', 'Trade Name']",Active
11,None,None,"Elite Golf 2, LLC",336584,"['X Golf Bismarck', 'Trade Name']",Active
21,None,None,"HAWLEY REALTY, INC.",240582,"['XTREME TROPHY PROPERTIES', 'Trade Name']",Active
29,None,None,"XTREME ELECTRICAL SERVICES, INC.",244002,"['X ELECTRIC', 'Trade Name']",Active


In [163]:
# creating a new worksheet
# date = datetime.today().strftime("%Y-%m-%d")
# worksheet = spreadsheet.add_worksheet(title=date,rows=150, cols=6)

In [144]:
com_reg_agent = df_prep.loc[df_prep.commercial_registered_agent.notnull()]
reg_agent = df_prep.loc[df_prep.registered_agent.notnull()]
owners = df_prep.loc[df.owners.notnull()]

rows_com_reg_agent,_ = com_reg_agent.shape
rows_reg_agent,_ = com_reg_agent.shape
rows_owners,_ = com_reg_agent.shape